<a href="https://colab.research.google.com/github/Goodwill-Khoa/219080534/blob/main/INDABAX2022g3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Possible imports PLeasee check with functions as well
!pip install face_recognition

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [37]:
!pip install imutils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install os

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


In [8]:
import imutils
import numpy as np
import cv2
#import face_recognition
import os
from google.colab.patches import cv2_imshow
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

In [9]:
from google.colab import files
files.upload()

{}

In [ ]:
from zipfile import ZipFile
file_path = "TrainingKnonw.zip"

with ZipFile(file_path, 'r') as zip:
  zip.extractall()
  print("Done")

Done


In [2]:
#Known = Storage and application sheet. This will be the Photos and Names of the user
# We ill just load an existing known folders. Seach folder will contains photos ad a names 

#this will just be photos with names and no names 

#We can load the Image from a source

def loadImage(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename			# this is a path to the image


In [4]:
image2path = loadImage()  # we will save this path so we can use it again

NameError: ignored

In [12]:
# now we will do the testing
# we opted to use a function that we can call at any moment

def IdentifyFace(path):
	import face_recognition
	import cv2
	import os
	from google.colab.patches import cv2_imshow
        
	def read_img(path):
		img = cv2.imread(path)
		(h, w) = img.shape[:2]
		width = 500
		ratio = width / float(w)
		height = int(h * ratio)
		return cv2.resize(img, (width, height))


	known_encodings = []
	known_names = []
	known_dir = 'Training knonw'

	for file in os.listdir(known_dir):
		img = read_img(known_dir + '/' + file)
		img_enc = face_recognition.face_encodings(img)[0]
		known_encodings.append(img_enc)
		known_names.append(file.split('.')[0])

	unknown_dir = 'unknown'
	for file in os.listdir(unknown_dir):
		print("Processing", file)
		img = read_img(unknown_dir + '/' + file)
		img_enc = face_recognition.face_encodings(img)[0]

		results = face_recognition.compare_faces(known_encodings, img_enc)
		# print(face_recognition.face_distance(known_encodings, img_enc))

		for i in range(len(results)):
			if results[i]:
				name = known_names[i]
				(top, right, bottom, left) = face_recognition.face_locations(img)[0]
				cv2.rectangle(img, (left, top), (right, bottom), (0, 0, 255), 2)
				cv2.putText(img, name, (left+2, bottom+20), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1)
				cv2_imshow(img)
            
			#print(results)
	return(results) # we just want to collect the name

In [1]:
Person1Name = IdentifyFace(image2path)

NameError: ignored

In [ ]:
print(Person1Name)